**Homework main task**: overcome Mistral hallucinations in chat mode.

### Plan:

- Test the pre-trained model as an online store assistant.
- Completely train the model using the LoRA method on data from chatbot communication with clients on various products.
- Test the completed model.

:You need to fill in all the blankes in the code (there are 7 spaces in total).

**Note!** Please safe all cells outputs, otherwise the task will not be accepted!

# Fine-tune Mistral 7B.

In [ ]:
!pip install peft transformers bitsandbytes accelerate trl datasets -U -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

Let's first test how the Mistral 7b, pre-trained in Russian (the original model was trained only in English), will cope with this task. Let the model play the role of an assistant for an online smartphone store.

In [ ]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
RESPONSE_TEMPLATE = "<s>bot\n"
SYSTEM_PROMPT = """Ты – чат-бот технической поддержки Xiaomi Store, который помогает клиенту выбрать наиболее подходящий для него смартфон. Опираясь на описание смартфонов, помоги клиенту выбрать наиболее подходящий для него смартфон. Если ответа на вопрос клиента нет в приведенном описании, ответь "У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram".
Описание смартфонов из интернет-магазина сотовой связи Xiaomi Store:
1. Смартфон Xiaomi Redmi Note 10 Pro: Этот смартфон оснащен дисплеем Super AMOLED с разрешением 1080 x 2400 пикселей, что обеспечивает четкую и яркую картинку. Он также имеет камеру на 64 Мп с возможностью записи видео в 4K. Процессор Snapdragon 732G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
2. Смартфон Xiaomi Mi 11 Lite: Этот смартфон имеет ультратонкий и легкий дизайн, который удобно держать в руке. Он оснащен 6.55-дюймовым AMOLED-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп способна делать качественные фотографии, а быстрый процессор Qualcomm Snapdragon 732G позволяет работать с приложениями плавно.
3. Смартфон Xiaomi Redmi 9T: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей, который обеспечивает реалистичную цветопередачу. Камера на 48 Мп делает четкие фото, а процессор Snapdragon 662 обеспечивает быструю работу. Аккумулятор на 6000 мАч позволяет использовать устройство долгое время без подзарядки.
4. Смартфон Xiaomi Mi 10T Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 108 Мп, что позволяет делать потрясающие фотографии. Процессор Qualcomm Snapdragon 865 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы.
5. Смартфон Xiaomi Redmi Note 9 Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп с поддержкой искусственного интеллекта позволяет делать яркие и четкие фотографии. Процессор Snapdragon 720G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
6. Смартфон Xiaomi Mi 10T Lite: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 64 Мп, что позволяет делать яркие и детальные фотографии. Процессор Qualcomm Snapdragon 750G ускоряет работу с приложениями, а аккумулятор на 4820 мАч обеспечивает долгую автономность.
7. Смартфон Xiaomi Redmi Note 8 Pro: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 64 Мп делает четкие и яркие фото, а процессор MediaTek Helio G90T обеспечивает плавную работу. Аккумулятор на 4500 мАч достаточно емкий для долгого использования.
8. Смартфон Xiaomi Mi 10 Pro: Этот смартфон оснащен 6.67-дюймовым AMOLED-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 108 Мп и оптическая стабилизация изображения позволяют делать высококачественные фотографии. Процессор Snapdragon 865 ускоряет работу с приложениями, а аккумулятор на 4500 мАч достаточно емкий.
9. Смартфон Xiaomi Redmi 9C: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 720 x 1600 пикселей. Он имеет камеру на 13 Мп, которая делает четкие фото в хороших условиях освещения. Процессор MediaTek Helio G35 обеспечивает достаточную производительность для базовых задач, а аккумулятор на 5000 мАч обеспечивает долгое время работы.
10. Смартфон Xiaomi Mi 11 Ultra: Этот смартфон оснащен 6.81-дюймовым AMOLED-дисплеем с разрешением 1440 x 3200 пикселей. Он имеет камеру на 50 Мп, а также вспомогательные камеры для различных эффектов съемки. Процессор Snapdragon 888 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы."""

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=MESSAGE_TEMPLATE,
        system_prompt=SYSTEM_PROMPT,
        response_template=RESPONSE_TEMPLATE
    ):
        # Инициализация класса Conversation с шаблонами сообщений и системным запросом.
        # message_template - шаблон для каждого сообщения (например, форматирование текста).
        # system_prompt - начальный запрос, задающий контекст разговора.
        # response_template - шаблон для ответа, который будет добавлен в конце текста.

        self.message_template = message_template  # Сохранение шаблона сообщений.
        self.response_template = response_template  # Сохранение шаблона ответа.
        # Инициализация списка сообщений с первым сообщением от системы, определяющим контекст беседы.
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        # Метод для добавления сообщения пользователя в список сообщений.
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        # Метод для добавления сообщения бота в список сообщений.
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        # Метод для формирования финального текста запроса, который будет отправлен для генерации ответа.
        final_text = ""  # Инициализация переменной для хранения окончательного текста.
        for message in self.messages:
            # Форматирование каждого сообщения в соответствии с message_template.
            message_text = self.message_template.format(**message)
            final_text += message_text  # Добавление отформатированного текста к итоговому.

        final_text += RESPONSE_TEMPLATE  # Добавление шаблона ответа в конце текста.
        return final_text.strip()  # Возвращение итогового текста без лишних пробелов в начале и в конце.


In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    # Функция для генерации текста на основе заданного промпта с использованием модели и токенизатора.

    # Токенизация промпта: превращаем текстовый промпт в числовые представления (токены),
    # которые понимает модель. Параметр return_tensors="pt" означает, что данные
    # будут возвращены в формате PyTorch тензоров. add_special_tokens=False указывает,
    # что специальные токены (например, [CLS], [SEP]) не будут добавлены.
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)

    # Переносим данные на устройство, на котором работает модель (например, GPU или CPU).
    data = {k: v.to(model.device) for k, v in data.items()}

    # Генерация выходных идентификаторов (токенов) с использованием модели.
    # Параметр generation_config задает конфигурацию генерации, например,
    # максимальную длину ответа, температуру и т.д.
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]

    # Обрезаем выходные идентификаторы, чтобы исключить исходные токены промпта.
    # Таким образом, оставляем только сгенерированный моделью ответ.
    output_ids = output_ids[len(data["input_ids"][0]):]

    # Декодируем токены обратно в текст, исключая специальные токены, такие как [PAD] или [SEP].
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

    # Возвращаем итоговый сгенерированный текст, убирая лишние пробелы в начале и в конце.
    return output.strip()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [ ]:
# Загружаем конфигурацию модели PeftConfig из предобученной модели, используя ее имя.
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем предобученную модель для задачи автозавершения текста (Causal Language Modeling).
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,  # Используем базовую модель из конфигурации.
    load_in_8bit=False,  # Загружаем модель с точностью float16, а не 8-битную версию.
    torch_dtype=torch.float16,  # Определяем тип данных тензоров как 16-битные числа с плавающей запятой (fp16).
    device_map="auto",  # Автоматически распределяем модель по доступным устройствам (например, на GPU, если он доступен).
)

model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float16)

# Переводим модель в режим оценки, чтобы отключить функции, используемые только при обучении,
# такие как dropout, и настроить модель на генерацию предсказаний.
model.eval()


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [ ]:
conversation = Conversation()

In [ ]:
questions = [
    "Какой процессор у смартфона Xiaomi Redmi Note 14 Pro 5G?", # MediaTek Dimensity 7300 Ultra
    "Какое вес у смартфона Xiaomi Redmi Note 13 5G?", # 173.5 г
    "Какая видеокарта у ноутбука Xiaomi RedmiBook 13?" # GeForce MX250
]

for user_uttr in questions:
    print(f"Клиент: {user_uttr}")
    conversation.add_user_message(user_uttr)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    output = output.split("bot")[0].strip()
    print(f"Чат-бот: {output}")

Клиент: Какой процессор у смартфона Xiaomi Redmi Note 14 Pro 5G?
Чат-бот: Процессор Xiaomi Redmi Note 14 Pro 5G оснащен процессором MediaTek Dimensity 920.
Клиент: Какое вес у смартфона Xiaomi Redmi Note 13 5G?
Чат-бот: Вес смартфона Xiaomi Redmi Note 13 Pro 5G составляет около 190 граммов.
Клиент: Какая видеокарта у ноутбука Xiaomi RedmiBook 13?
Чат-бот: Ваш вопрос не относится к смартфонам, а к ноутбукам. У ноутбуков Xiaomi RedmiBook 13 используются графические адаптеры Intel UHD Graphics или NVIDIA GeForce MX450.


Let's start a conversation. Try asking to tell about tablets that have 4K video quality. We see that the bot starts to invent non-existent information, that is, it starts to hallucinate.

Downloading data for additional fine-tuning.

In [ ]:
import gc

# очищаем память
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_xKwkNFisKzDQ1CPPDqvKWdPRzZvO4MT' -O company_cases.json

--2025-04-12 06:00:31--  https://docs.google.com/uc?export=download&id=1_xKwkNFisKzDQ1CPPDqvKWdPRzZvO4MT
Resolving docs.google.com (docs.google.com)... 142.250.125.113, 142.250.125.101, 142.250.125.100, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1_xKwkNFisKzDQ1CPPDqvKWdPRzZvO4MT&export=download [following]
--2025-04-12 06:00:31--  https://drive.usercontent.google.com/download?id=1_xKwkNFisKzDQ1CPPDqvKWdPRzZvO4MT&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.126.132, 2607:f8b0:4001:c1d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.126.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70497894 (67M) [application/octet-stream]
Saving to: ‘company_cases.json’

company_cases.json  100%[===================>]  67.23M   165M

In [ ]:
import json
import torch
from datasets import Dataset
from peft import LoraConfig
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
with open("company_cases.json", 'r') as inp:
    raw_dataset = json.load(inp)

# пришлось оставить только короткие sequenc'ы, так как в противном случае случается CUDA out of memory
# не получилось использовать кластер на cloud.ru, он уходил в бесконечную загрузку
raw_dataset = [line for line in raw_dataset if len(str(line)) <= 2**11*1.5]

train_raw_dataset, test_raw_dataset = train_test_split(raw_dataset)
train_test_raw_dataset = {"train": train_raw_dataset, "test": test_raw_dataset}

Let's convert the dataset into the format used in the Dataset class.

In [ ]:
raw_dataset_dict = {}
for data_type in ["train", "test"]:
    raw_dataset_dict[data_type] = {
        "instruction": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "personality": [element['personality'] for element in train_test_raw_dataset[data_type]],
        "context": [element['context'] for element in train_test_raw_dataset[data_type]],
        "dialog_start_line": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "dialog": [element['dialog'] for element in train_test_raw_dataset[data_type]]
    }

train_dataset = Dataset.from_dict(raw_dataset_dict["train"])
test_dataset = Dataset.from_dict(raw_dataset_dict["test"])

Function for formatting the prompt.

In [ ]:
def formatting_prompt_func(example):
    prompt = f"<s>system\n{example['instruction']}"
    if example['personality']:
        prompt += f"\n{example['personality']}"
    prompt += f"\n{example['context']}"
    if example['dialog_start_line']:
        prompt += f"\n{example['dialog_start_line']}"
    prompt += "</s>"
    for element in example['dialog']:
        prompt += f"<s>{element['role']}\n{element['content']}</s>"
    return prompt

For additional training, we will take not the ready Russian-language Mistral, but the pre-trained Mistral on the Open-Orca dataset.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")
tokenizer.pad_token_id = 0

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Let's make the gradient during training flow only through the tokens of the last replica of the chatbot, and let's make the labels for the remaining tokens equal to -100.

In [ ]:
response_template = "bot\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# Будем обучать модель в int4 для уменьшения требуемой видеопамяти

model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca",
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=BitsAndBytesConfig(load_in_4bit=True)
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing LoraConfig.

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.075,
    target_modules=[
        "lm_head",
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

Setting parameters for additional training.

In [ ]:
training_arguments = TrainingArguments(
    output_dir="mistral7b_lora",
    num_train_epochs=3,
    save_total_limit=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    weight_decay=0.0005,
    dataloader_drop_last=True,
    fp16=True,
    logging_steps=50,
    learning_rate=3e-5,
    lr_scheduler_type="cosine",
    report_to=None
)

Disable tokenizer parallelism to avoid deadlocks.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import warnings

# убираем вывод warning'ов
warnings.filterwarnings("ignore")
# отключаем wandb
os.environ["WANDB_MODE"] = "disabled"

Starting fine-tuning.

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    formatting_func=formatting_prompt_func,
    data_collator=collator,
    peft_config=peft_config,
    args=training_arguments
)

trainer.train()

Applying formatting function to train dataset:   0%|          | 0/3182 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/3182 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/3182 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3182 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3182 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/1061 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1061 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
50,0.941800
100,0.767800
150,0.708500
200,0.658300
250,0.667400
300,0.642100
350,0.627600
400,0.633500
450,0.529700
500,0.520900


TrainOutput(global_step=1191, training_loss=0.5276919468223898, metrics={'train_runtime': 4365.7049, 'train_samples_per_second': 2.187, 'train_steps_per_second': 0.273, 'total_flos': 3.784782277818304e+17, 'train_loss': 0.5276919468223898})

Test the resulting model.

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

Loading the adapter checkpoint.

In [ ]:
BASE_MODEL = "Open-Orca/Mistral-7B-OpenOrca"
ADAPTER_MODEL = "mistral7b_lora/checkpoint-1191"
MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
RESPONSE_TEMPLATE = "<s>bot\n"
SYSTEM_PROMPT = """Ты – чат-бот технической поддержки Xiaomi Store, который помогает клиенту выбрать наиболее подходящий для него смартфон. Опираясь на описание смартфонов, помоги клиенту выбрать наиболее подходящий для него смартфон. Если ответа на вопрос клиента нет в приведенном описании, ответь "У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram".{personality}
Описание смартфонов из интернет-магазина сотовой связи Xiaomi Store:
1. Смартфон Xiaomi Redmi Note 10 Pro: Этот смартфон оснащен дисплеем Super AMOLED с разрешением 1080 x 2400 пикселей, что обеспечивает четкую и яркую картинку. Он также имеет камеру на 64 Мп с возможностью записи видео в 4K. Процессор Snapdragon 732G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
2. Смартфон Xiaomi Mi 11 Lite: Этот смартфон имеет ультратонкий и легкий дизайн, который удобно держать в руке. Он оснащен 6.55-дюймовым AMOLED-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп способна делать качественные фотографии, а быстрый процессор Qualcomm Snapdragon 732G позволяет работать с приложениями плавно.
3. Смартфон Xiaomi Redmi 9T: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей, который обеспечивает реалистичную цветопередачу. Камера на 48 Мп делает четкие фото, а процессор Snapdragon 662 обеспечивает быструю работу. Аккумулятор на 6000 мАч позволяет использовать устройство долгое время без подзарядки.
4. Смартфон Xiaomi Mi 10T Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 108 Мп, что позволяет делать потрясающие фотографии. Процессор Qualcomm Snapdragon 865 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы.
5. Смартфон Xiaomi Redmi Note 9 Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп с поддержкой искусственного интеллекта позволяет делать яркие и четкие фотографии. Процессор Snapdragon 720G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
6. Смартфон Xiaomi Mi 10T Lite: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 64 Мп, что позволяет делать яркие и детальные фотографии. Процессор Qualcomm Snapdragon 750G ускоряет работу с приложениями, а аккумулятор на 4820 мАч обеспечивает долгую автономность.
7. Смартфон Xiaomi Redmi Note 8 Pro: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 64 Мп делает четкие и яркие фото, а процессор MediaTek Helio G90T обеспечивает плавную работу. Аккумулятор на 4500 мАч достаточно емкий для долгого использования.
8. Смартфон Xiaomi Mi 10 Pro: Этот смартфон оснащен 6.67-дюймовым AMOLED-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 108 Мп и оптическая стабилизация изображения позволяют делать высококачественные фотографии. Процессор Snapdragon 865 ускоряет работу с приложениями, а аккумулятор на 4500 мАч достаточно емкий.
9. Смартфон Xiaomi Redmi 9C: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 720 x 1600 пикселей. Он имеет камеру на 13 Мп, которая делает четкие фото в хороших условиях освещения. Процессор MediaTek Helio G35 обеспечивает достаточную производительность для базовых задач, а аккумулятор на 5000 мАч обеспечивает долгое время работы.
10. Смартфон Xiaomi Mi 11 Ultra: Этот смартфон оснащен 6.81-дюймовым AMOLED-дисплеем с разрешением 1440 x 3200 пикселей. Он имеет камеру на 50 Мп, а также вспомогательные камеры для различных эффектов съемки. Процессор Snapdragon 888 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы."""

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=MESSAGE_TEMPLATE,
        system_prompt=SYSTEM_PROMPT,
        response_template=RESPONSE_TEMPLATE,
        personality=None
    ):
        self.message_template = message_template
        self.response_template = response_template
        if personality:
            system_prompt = system_prompt.format(personality=personality)
        else:
            system_prompt = system_prompt.format(personality="")
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += RESPONSE_TEMPLATE
        return final_text.strip()

In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
tokenizer.pad_token_id = 0
generation_config = GenerationConfig(
    pad_token_id=0,
    bos_token_id=1,
    eos_token_id=2,
    temperature=0.2,
    top_p=0.9,
    top_k=45,
    max_length=2**11,
    do_sample=True,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, torch_dtype=torch.float16)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.075, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
   

In [ ]:
conversation = Conversation()

We start the dialogue again and ask about the tablets. Were you able to overcome the hallucinations?

In [ ]:
questions = [
    "Какой процессор у смартфона Xiaomi Redmi Note 14 Pro 5G?", # MediaTek Dimensity 7300 Ultra
    "Какое вес у смартфона Xiaomi Redmi Note 13 5G?", # 173.5 г
    "Какая видеокарта у ноутбука Xiaomi RedmiBook 13?" # GeForce MX250
]

for user_uttr in questions:
    print(f"Клиент: {user_uttr}")
    conversation.add_user_message(user_uttr)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    output = output.split("bot")[0].strip()
    print(f"Чат-бот: {output}")

Клиент: Какой процессор у смартфона Xiaomi Redmi Note 14 Pro 5G?
Чат-бот: У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram. </s>
Клиент: Какое вес у смартфона Xiaomi Redmi Note 13 5G?
Чат-бот: У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram. </s>
Клиент: Какая видеокарта у ноутбука Xiaomi RedmiBook 13?
Чат-бот: У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram. </s>
